In [2]:
#############----------2.Splitting data into 70% Train and 30% test dataset------------------################
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345 )
X_train.shape
df_train=X_train.copy()
df_train['T0_txn_amt']=y_train


In [1763]:
#############----------3.Data Transforamtion on training dataset-----############################

#1.All categorical variables are encoded
def cat_feature_engineering (var):
    global X_train
    var_def=pd.get_dummies(X_train[var], dummy_na = True, drop_first = True, prefix = 'R1_'+var)
    X_train = pd.merge(X_train, var_def, left_index=True, right_index=True)

cat_list=['Product','imsstatus','monthlyfees']
for i in cat_list:
    cat_feature_engineering(i)

#2.Creating new intermediate variables
X_train['amt_diff']=X_train['T1_txn_amt']-X_train['T2_txn_amt']
X_train['cnt_diff']=(X_train['T1_txn_cnt']-X_train['T2_txn_cnt'])
X_train.drop(columns=['imsstatus','monthlyfees','Product','R1_monthlyfees_nan','T2_txn_amt','T2_txn_cnt','R1_imsstatus_nan'], inplace=True)
X_train['amt_posms']=X_train.R1_Product_POSMS*X_train.T1_txn_amt
X_train.drop(columns=['R1_Product_POSMS'], inplace=True)


In [1764]:
#########---------------4.multicollinearity check---------------###########
X_train.corr()

,T1_txn_cnt,T1_txn_amt,days_since_open,R1_Product_QBMS,R1_Product_QBO,R1_Product_nan,R1_imsstatus_REOPENED,R1_monthlyfees_Paid,amt_diff,cnt_diff,amt_posms
T1_txn_cnt,1.000000,0.423425,0.188837,0.085776,-0.196164,-0.036518,0.005302,0.343540,0.037015,0.078919,0.267937
T1_txn_amt,0.423425,1.000000,0.121001,0.124164,-0.092359,-0.030513,0.009641,0.211706,0.098673,-0.021493,0.273724
days_since_open,0.188837,0.121001,1.000000,0.378558,-0.496132,-0.017338,0.118685,0.538720,-0.033313,-0.054585,0.014135
R1_Product_QBMS,0.085776,0.124164,0.378558,1.000000,-0.637170,-0.158120,0.047679,0.365264,-0.025076,-0.009603,-0.060357
R1_Product_QBO,-0.196164,-0.092359,-0.496132,-0.637170,1.000000,-0.242095,-0.065598,-0.382058,0.012498,0.023541,-0.092412
R1_Product_nan,-0.036518,-0.030513,-0.017338,-0.158120,-0.242095,1.000000,0.007279,-0.006784,0.002798,0.006866,-0.022933
R1_imsstatus_REOPENED,0.005302,0.009641,0.118685,0.047679,-0.065598,0.007279,1.000000,0.081129,-0.004695,0.001330,-0.001460
R1_monthlyfees_Paid,0.343540,0.211706,0.538720,0.365264,-0.382058,-0.006784,0.081129,1.000000,-0.027280,-0.067903,0.130822
amt_diff,0.037015,0.098673,-0.033313,-0.025076,0.012498,0.002798,-0.004695,-0.027280,1.000000,0.296091,0.037446
cnt_diff,0.078919,-0.021493,-0.054585,-0.009603,0.023541,0.006866,0.001330,-0.067903,0.296091,1.000000,-0.041461


In [1765]:
#############----------5.Running Model-----############################
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
type(lr)
lr.fit(X_train, df_train.T0_txn_amt)
list(zip(X_train.columns, lr.coef_))

[('T1_txn_cnt', 133.57048178006784),
 ('T1_txn_amt', 1.0026952114965781),
 ('days_since_open', -0.05077602069224053),
 ('R1_Product_QBMS', 699.9733008458857),
 ('R1_Product_QBO', 345.26761443120256),
 ('R1_Product_nan', 160.70898114542717),
 ('R1_imsstatus_REOPENED', -116.3090536202373),
 ('R1_monthlyfees_Paid', 525.2826861416629),
 ('amt_diff', -0.2970321311005884),
 ('cnt_diff', 110.5065054792402),
 ('amt_posms', 0.16236225780164412)]

In [1766]:
#############----------6.Data Transforamtion on testing dataset-----############################
df_test=X_test.copy()
df_test['T0_txn_amt']=y_test


#1.# All categorical variables are encoded
def cat_feature_engineering (var):
    global X_test
    var_def=pd.get_dummies(X_test[var], dummy_na = True, drop_first = True, prefix = 'R1_'+var)
    X_test = pd.merge(X_test, var_def, left_index=True, right_index=True)

cat_list=['Product','imsstatus','monthlyfees']
for i in cat_list:
    cat_feature_engineering(i)

#2.Creating new intermediate variables
X_test['amt_diff']=X_test['T1_txn_amt']-X_test['T2_txn_amt']
X_test['cnt_diff']=(X_test['T1_txn_cnt']-X_test['T2_txn_cnt'])
X_test.drop(columns=['imsstatus','monthlyfees','Product','T2_txn_cnt','T2_txn_amt','R1_monthlyfees_nan','R1_imsstatus_nan'], inplace=True)
X_test['amt_posms']=X_test.R1_Product_POSMS*X_test.T1_txn_amt
X_test.drop(columns=['R1_Product_POSMS'], inplace=True)






In [1767]:
#############----------7.Predicting values-----############################
y_pred=lr.predict(X_test)

In [1768]:
#############----------8.RMSE-----###############################
from sklearn import metrics
np.sqrt(metrics.mean_squared_error(df_test.T0_txn_amt, y_pred))

11907.25072672285

In [1769]:
#############----------9.R-Squared-----############################
metrics.r2_score(df_test.T0_txn_amt, y_pred)

0.8043443716356057

In [ ]:
#Summary
#.Able to reduce RMSE to 11907.25 and increase R-Squared to 0.804344.
